In [1]:
from __future__ import print_function


import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from skimage.io import imshow
from keras import backend as K
from keras.constraints import Constraint
from keras.datasets import fashion_mnist
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


Using TensorFlow backend.


In [0]:
def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):

    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == np.argmax(label_vec1[num]):
            acc1 += 1

    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == np.argmax(label_vec2[num]):
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)

    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )


    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('maximum inference accuracy is:', max_accuracy)
    print('precision is:', best_precision)
    print('recall is:', best_recall)
    return max_accuracy


In [0]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}


def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/bangkok" 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
print("Loading dataset...")
data_set =np.genfromtxt(path,delimiter=',')
print("Dataset loaded.")

Loading dataset...
Dataset loaded.


In [0]:
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, precision_score, recall_score

X = data_set[:,1:].astype(np.float64)
Y = (data_set[:,0]).astype(np.int32)-1


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.45)

y_train = keras.utils.to_categorical(y_train, 30)
y_test = keras.utils.to_categorical(y_test, 30)


In [7]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(446,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(30, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               228864    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 30)                3870      
Total params: 396,958
Trainable params: 396,958
Non-trainable params: 0
_________________________________________________________________




In [8]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 2755 samples, validate on 2255 samples
Epoch 1/50





2755/2755 [==============================] - 20s 7ms/step - loss: 2.5686 - acc: 0.2766 - val_loss: 2.5679 - val_acc: 0.1871
Epoch 2/50
2755/2755 [==============================] - 0s 71us/step - loss: 1.2747 - acc: 0.6312 - val_loss: 2.5400 - val_acc: 0.3078
Epoch 3/50
2755/2755 [==============================] - 0s 80us/step - loss: 0.7890 - acc: 0.7746 - val_loss: 1.8929 - val_acc: 0.4882
Epoch 4/50
2755/2755 [==============================] - 0s 71us/step - loss: 0.5143 - acc: 0.8497 - val_loss: 1.5239 - val_acc: 0.5614
Epoch 5/50
2755/2755 [==============================] - 0s 74us/step - loss: 0.3212 - acc: 0.9169 - val_loss: 1.3814 - val_acc: 0.5867
Epoch 6/50
2755/2755 [==============================] - 0s 72us/step - loss: 0.2180 - acc: 0.9474 - val_loss: 2.0671 - val_acc: 0.5366
Epoch 7/50
2755/2755 [==================

In [9]:
output_train=model.predict(x_train)
output_test=model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (1.0, 0.6350332594235033)
maximum inference accuracy is: 0.7832507716266734
precision is: 0.6157233455093559
recall is: 1.5070780399274046
Maximum Accuracy: 0.7832507716266734


# **Sparse**

In [28]:
masks = create_sparsity_masks(model,90)#Closest 30% to 0

sparse_model = Sequential()
sparse_model.add(Dense(512,kernel_constraint=Sparse(masks[0]), activation='relu', input_shape=(446,)))
sparse_model.add(Dense(256,kernel_constraint=Sparse(masks[1]), activation='relu'))
sparse_model.add(Dense(128,kernel_constraint=Sparse(masks[2]), activation='relu'))
sparse_model.add(Dense(30,kernel_constraint=Sparse(masks[3]), activation='softmax'))

sparse_model.summary()

adam = Adam()
sparse_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
sparse_model.summary()
#Get weights from densely trained model
sparse_model.set_weights(model.get_weights())


# Train Sparse Model
sparse_model.fit(x_train,y_train,
          batch_size=32,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test))



Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 512)               228864    
_________________________________________________________________
dense_42 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_43 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_44 (Dense)             (None, 30)                3870      
Total params: 396,958
Trainable params: 396,958
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 512)               228864    
________________________

In [29]:
output_train=sparse_model.predict(x_train)
output_test=sparse_model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (1.0, 0.6350332594235033)
maximum inference accuracy is: 0.7952394557768381
precision is: 0.5992267682206228
recall is: 1.782940108892922
Maximum Accuracy: 0.7952394557768381
